<a href="https://colab.research.google.com/github/aqilzdhr/FloodForecasting/blob/main/Flood_Forecasting_Using_Weather_Parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Library

In [ ]:
# Tensorflow / Keras
from tensorflow import keras 
print('Tensorflow/Keras: %s' % keras.__version__) 
from keras.models import Sequential
from keras import Input 
from keras.layers import Dense, SimpleRNN

# Data manipulation
import pandas as pd 
print('pandas: %s' % pd.__version__) 
import numpy as np 
print('numpy: %s' % np.__version__) 
import math 

# Sklearn
import sklearn 
print('sklearn: %s' % sklearn.__version__) 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import MinMaxScaler 

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) 

Tensorflow/Keras: 2.9.0
pandas: 1.3.5
numpy: 1.21.6
sklearn: 1.0.2
plotly: 5.5.0


Data Manipulation

In [ ]:
pd.options.display.max_columns=100

from google.colab import files
uploaded = files.upload()

Saving data1.csv to data1.csv


In [ ]:
weather = "/content/data1.csv"
weather = pd.read_csv(weather)
weather.head(100)

,Date,UT time,Temperature (K),Relative Humidity (%),Pressure (hPa),Wind speed (m/s),Wind direction,Rainfall (mm),Flood
0,5/11/2015,24:00:00,299.27,89.13,974.84,1.01,97.75,74.591100,1
1,5/16/2015,24:00:00,299.25,87.39,975.16,1.06,147.34,62.907876,1
2,5/19/2015,24:00:00,298.74,86.76,975.74,1.32,212.52,61.708716,1
3,5/21/2015,24:00:00,298.87,86.45,976.07,0.43,311.13,114.142248,1
4,6/14/2015,24:00:00,298.82,84.94,973.93,0.19,236.35,109.579824,1
...,...,...,...,...,...,...,...,...,...
95,8/5/2019,24:00:00,298.64,82.48,974.72,1.40,207.01,2.447867,0
96,8/20/2019,24:00:00,299.77,81.86,974.28,0.42,161.48,62.227584,1
97,8/21/2019,24:00:00,297.34,87.43,975.79,1.56,254.68,63.547956,1
98,8/22/2019,24:00:00,298.46,83.97,976.17,0.20,132.76,12.019698,0


Data Preprocessing

In [ ]:
# Checking dataset data types
weather.dtypes

Date                      object
UT time                   object
Temperature (K)          float64
Relative Humidity (%)    float64
Pressure (hPa)           float64
Wind speed (m/s)         float64
Wind direction           float64
Rainfall (mm)            float64
Flood                      int64
dtype: object

In [ ]:
weather.shape

(106, 9)

In [ ]:
# Checking dataset info
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   106 non-null    object 
 1   UT time                106 non-null    object 
 2   Temperature (K)        106 non-null    float64
 3   Relative Humidity (%)  106 non-null    float64
 4   Pressure (hPa)         106 non-null    float64
 5   Wind speed (m/s)       106 non-null    float64
 6   Wind direction         106 non-null    float64
 7   Rainfall (mm)          106 non-null    float64
 8   Flood                  106 non-null    int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 7.6+ KB


In [ ]:
weather.describe()

,Temperature (K),Relative Humidity (%),Pressure (hPa),Wind speed (m/s),Wind direction,Rainfall (mm),Flood
count,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000
mean,298.589340,85.241509,975.475472,1.065283,158.877358,46.951045,0.500000
std,0.876973,4.311057,1.315303,0.636821,93.319096,47.220590,0.502375
min,296.620000,75.740000,971.930000,0.140000,6.160000,0.150296,0.000000
25%,297.985000,81.907500,974.467500,0.592500,67.862500,3.721841,0.000000
50%,298.685000,85.175000,975.625000,0.950000,151.220000,40.444260,0.500000
75%,299.220000,87.905000,976.220000,1.400000,245.910000,80.660025,1.000000
max,300.690000,94.160000,978.940000,3.250000,348.630000,192.352788,1.000000


In [ ]:
# Checking for null values in dataset
weather.isnull().any()

Date                     False
UT time                  False
Temperature (K)          False
Relative Humidity (%)    False
Pressure (hPa)           False
Wind speed (m/s)         False
Wind direction           False
Rainfall (mm)            False
Flood                    False
dtype: bool

In [ ]:
# Checking dataset columns
weather.columns

Index(['Date', 'UT time', 'Temperature (K)', 'Relative Humidity (%)',
       'Pressure (hPa)', 'Wind speed (m/s)', 'Wind direction', 'Rainfall (mm)',
       'Flood'],
      dtype='object')

In [ ]:
# Removing rows with null values
weather.dropna().isnull().sum()

Date                     0
UT time                  0
Temperature (K)          0
Relative Humidity (%)    0
Pressure (hPa)           0
Wind speed (m/s)         0
Wind direction           0
Rainfall (mm)            0
Flood                    0
dtype: int64

Data Training

In [ ]:
# Plotting Rainfall on Chart

# Plot rainfall
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather['Date'], 
                         y=weather['Rainfall (mm)'],
                         mode='lines',
                         name='Rainfall',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Date'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='mm'
                )

# Set title
fig.update_layout(title=dict(text="Rainfall (mm)", 
                             font=dict(color='black')))

Training and Evaluating RNN

In [ ]:
def prep_data(datain, time_step):
    # Setting y-array
    y_indices = np.arange(start=time_step, stop=len(datain), step=time_step)
    # Creating y array based on the above indices 
    y_tmp = datain[y_indices]
    
    # Setting x-array 
    rows_X = len(y_tmp)
    X_tmp = datain[range(time_step*rows_X)]

    X_tmp = np.reshape(X_tmp, (rows_X, time_step, 1))
    return X_tmp, y_tmp

In [ ]:

# Selecting data for modeling and applying Min Max Scaling
X=weather[['Rainfall (mm)']]
scaler = MinMaxScaler()
X_scaled=scaler.fit_transform(X)


# Create training and testing samples
train_data, test_data = train_test_split(X_scaled, test_size=0.2, shuffle=False)


# Preparing input X and target y arrays using previously defined function
time_step = 7
X_train, y_train = prep_data(train_data, time_step)
X_test, y_test = prep_data(test_data, time_step)


# Specifying the structure of a Neural Network
model = Sequential(name="Flood-Forecasting-RNN-Model") 
model.add(Input(shape=(time_step,1), name='Input-Layer')) 
model.add(SimpleRNN(units=1, activation='tanh', name='Hidden-Recurrent-Layer')) 
model.add(Dense(units=1, activation='tanh', name='Hidden-Layer')) 
model.add(Dense(units=1, activation='linear', name='Output-Layer')) 


# Compiling keras model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], 
              loss_weights=None, 
              weighted_metrics=None, 
              run_eagerly=None, 
              steps_per_execution=None
             )


# Fiting keras model on the dataset
model.fit(X_train,
          y_train, 
          batch_size=1, 
          epochs=20, 
          verbose='auto', 
          callbacks=None,
          validation_split=0.0, 
          shuffle=True, 
          class_weight=None,  
          sample_weight=None, 
          initial_epoch=0, 
          steps_per_epoch=None,  
          validation_steps=None, 
          validation_batch_size=None, 
          validation_freq=1, 
          max_queue_size=10, 
          workers=1, 
          use_multiprocessing=False, 
         )


# Using model to generate predictions

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)


# Model Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Note, the last parameter in each layer is bias while the rest are weights")
print("")
for layer in model.layers:
    print(layer.name)
    for item in layer.get_weights():
        print("  ", item)
print("")
print('---------- Evaluation on Training Data ----------')
print("MSE: ", mean_squared_error(y_train, pred_train))
print("")

print('---------- Evaluation on Test Data ----------')
print("MSE: ", mean_squared_error(y_test, pred_test))
print("")

Epoch 1/20
11/11 [==============================] - 1s 3ms/step - loss: 0.0526 - mean_squared_error: 0.0526 - mean_absolute_error: 0.1824
Epoch 2/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0484 - mean_squared_error: 0.0484 - mean_absolute_error: 0.1780    
Epoch 3/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0449 - mean_squared_error: 0.0449 - mean_absolute_error: 0.1771
Epoch 4/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0431 - mean_squared_error: 0.0431 - mean_absolute_error: 0.1759
Epoch 5/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0417 - mean_squared_error: 0.0417 - mean_absolute_error: 0.1768
Epoch 6/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0408 - mean_squared_error: 0.0408 - mean_absolute_error: 0.1781
Epoch 7/20
11/11 [==============================] - 0s 3ms/step - loss: 0.0404 - mean_squared_error: 0.0404 - mean_absolute_error: 0.1805
Epoch 8/20
11/11 [============

Plotting Actual and Predicted Rainfall on Chart

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(range(0,len(y_test))),
                         y=scaler.inverse_transform(y_test).flatten(),
                         mode='lines',
                         name='Rainfall - Actual (Test)',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))
fig.add_trace(go.Scatter(x=np.array(range(0,len(pred_test))),
                         y=scaler.inverse_transform(pred_test).flatten(),
                         mode='lines',
                         name='Rainfall - Predicted (Test)',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Observation'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='mm'
                )

# Set figure title
fig.update_layout(title=dict(text="Rainfall (mm)", 
                             font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )

fig.show()

Model Prediction

In [ ]:
X_every=weather[['Rainfall (mm)']]
X_every=scaler.transform(X_every)

for i in range(0, len(X_every)-time_step):
    if i==0:
        X_comb=X_every[i:i+time_step]
    else: 
        X_comb=np.append(X_comb, X_every[i:i+time_step])
X_comb=np.reshape(X_comb, (math.floor(len(X_comb)/time_step), time_step, 1))
print(X_comb.shape)

weather['Flood_prediction'] = np.append(np.zeros(time_step), scaler.inverse_transform(model.predict(X_comb)))

(99, 7, 1)
4/4 [==============================] - 0s 3ms/step


Plotting Prediction Results

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=weather['Date'],
                         y=weather['Rainfall (mm)'],
                         mode='lines',
                         name='Rainfall - Actual',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))
fig.add_trace(go.Scatter(x=weather['Date'],
                         y=weather['Flood_prediction'],
                         mode='lines',
                         name='Rainfall - Predicted',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Changing chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Updating axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Observation'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='mm'
                )

# Setting figure title
fig.update_layout(title=dict(text="Rainfall (mm)", 
                             font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )

fig.show()

In [ ]:
import streamlit as st
st.write ('Hello')
st.write('##run streamlit on colab with pyngrok')

ModuleNotFoundError: ignored